In [4]:
import nibabel as nb

data_dir = 'MRIsamples/'
data_dir_save = data_dir + 'hdr_to_nii/'
data_ext = '.hdr'
data_ext_save = '.nii'
filenames = ['4_8', '8_4', '100_23', '205_3']

for filename in filenames:
    try:
        #Se carga la imagen MRI simple
        path = data_dir + filename + data_ext
        img = nb.load(path)

        #Se muestra la información del encabezado
        print(img.header)
        print('*********************************************************************')

        #Se guarda la imagen MRI en formao NIFTI
        path = path.replace(data_dir, data_dir_save).replace(data_ext, data_ext_save)
        nb.save(img, path)
    except FileNotFoundError:
        print(f"No se pudo cargar el archivo: '{filename + data_ext}'")
        print('*********************************************************************')
    except AttributeError:
        print(f"El archivo: '{filename + data_ext}', se encuentra vacio")
        print('*********************************************************************')

<class 'nibabel.spm2analyze.Spm2AnalyzeHeader'> object, endian='<'
sizeof_hdr     : 348
data_type      : b'dsr      '
db_name        : b'..\\work_t1\\ibsr\\o'
extents        : 0
session_error  : 0
regular        : b'r'
hkey_un0       : b'0'
dim            : [  4 256  61 256   1   0   0   0]
vox_units      : b'mm'
cal_units      : b''
unused1        : 0
datatype       : int16
bitpix         : 16
dim_un0        : 0
pixdim         : [0.  1.  3.1 1.  0.  0.  0.  0. ]
vox_offset     : 0.0
scl_slope      : nan
scl_inter      : 0.0
funused3       : 0.0
cal_max        : 0.0
cal_min        : 0.0
compressed     : 0
verified       : 0
glmax          : 32767
glmin          : 0
descrip        : b'spm compatible'
aux_file       : b'none                   '
orient         : b''
origin         : [91 34 91  0  0]
generated      : b''
scannum        : b''
patient_id     : b''
exp_date       : b''
exp_time       : b''
hist_un0       : b''
views          : 0
vols_added     : 0
start_field    : 0
field_ski

In [5]:
pip install nibabel

^C
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import nibabel as nb

data_dir = 'MRIsamples/'
data_dir_save = data_dir + 'hdr_to_nii/'
data_ext = '.hdr'
data_ext_save = '.nii'

img = nb.load(data_dir+'4_8.img')
hdr = nb.load(data_dir+'4_8.hdr')

print(hdr.header)

<class 'nibabel.spm2analyze.Spm2AnalyzeHeader'> object, endian='<'
sizeof_hdr     : 348
data_type      : b'dsr      '
db_name        : b'..\\work_t1\\ibsr\\o'
extents        : 0
session_error  : 0
regular        : b'r'
hkey_un0       : b'0'
dim            : [  4 256  61 256   1   0   0   0]
vox_units      : b'mm'
cal_units      : b''
unused1        : 0
datatype       : int16
bitpix         : 16
dim_un0        : 0
pixdim         : [0.  1.  3.1 1.  0.  0.  0.  0. ]
vox_offset     : 0.0
scl_slope      : nan
scl_inter      : 0.0
funused3       : 0.0
cal_max        : 0.0
cal_min        : 0.0
compressed     : 0
verified       : 0
glmax          : 32767
glmin          : 0
descrip        : b'spm compatible'
aux_file       : b'none                   '
orient         : b''
origin         : [91 34 91  0  0]
generated      : b''
scannum        : b''
patient_id     : b''
exp_date       : b''
exp_time       : b''
hist_un0       : b''
views          : 0
vols_added     : 0
start_field    : 0
field_ski

In [38]:
import nibabel as nb
import numpy as np

data_dir = 'MRIsamples/'
data_dir_save = data_dir + 'hdr_to_nii/'
data_ext = '.hdr'
data_ext_save = '.nii'

img = nb.load(data_dir+'4_8.img')
hdr = nb.load(data_dir+'4_8.hdr')

nii = nb.nifti1.Nifti1Image(np.ones_like(np.asarray(img.dataobj)), hdr.affine, hdr.header)
print(nii)

<class 'nibabel.nifti1.Nifti1Image'>
data shape (256, 61, 256, 1)
affine: 
[[  -1.            0.            0.           90.        ]
 [   0.            3.0999999     0.         -102.29999542]
 [   0.            0.            1.          -90.        ]
 [   0.            0.            0.            1.        ]]
metadata:
<class 'nibabel.nifti1.Nifti1Header'> object, endian='<'
sizeof_hdr      : 348
data_type       : b'dsr      '
db_name         : b'..\\work_t1\\ibsr\\o'
extents         : 0
session_error   : 0
regular         : b'r'
dim_info        : 0
dim             : [  4 256  61 256   1   1   1   1]
intent_p1       : 0.0
intent_p2       : 0.0
intent_p3       : 0.0
intent_code     : none
datatype        : int16
bitpix          : 16
slice_start     : 0
pixdim          : [-1.   1.   3.1  1.   0.   1.   1.   1. ]
vox_offset      : 0.0
scl_slope       : nan
scl_inter       : nan
slice_end       : 0
slice_code      : unknown
xyzt_units      : 0
cal_max         : 0.0
cal_min         : 0.0
s